In [1]:
from ollama import Client

In [8]:
message = {'role': 'user', 'content': 'why is ice dark?'}
client = Client(base_url="http://127.0.0.1:8008")

In [9]:
response = client.chat(model='phi', messages=[message])

In [10]:
response

{'model': 'phi',
 'created_at': '2024-01-18T11:29:11.812362341Z',
 'message': {'role': 'assistant',
  'content': " Ice appears dark because it reflects very little light, while its surface absorbs most of it. When sunlight hits the surface of ice, some of the energy is absorbed by the molecules in the ice, which then re-emits that energy as heat. This process causes the ice to appear dark and dull, unlike snow or other types of white surfaces that reflect more light back into the environment.\n\nAdditionally, since ice is made up of frozen water molecules arranged in a crystalline structure, it has a relatively low refractive index compared to liquids like water. Refraction refers to how much light bends when it passes through a medium, and ice's lower refractive index causes it to scatter more light, giving the appearance of darkness or lack of color.\n\nIn summary, the dark appearance of ice is due to its absorption of light energy and scattering properties.\n"},
 'done': True,
 'tot

In [14]:
from datasets import IterableDataset

In [68]:
import json
import random

In [73]:
with open("vocab.json", "rb") as f:
    vocab_data = json.load(f)
ALL_TOKENS = [token for token in vocab_data.keys() if len(token)>3]
ALL_TOKENS = [token[1:] for token in ALL_TOKENS if token.startswith("Ġ")]

def get_random_token():
    return random.choice(ALL_TOKENS)

In [74]:
def data_from_phi():
    word = get_random_token()
    message = {'role': 'user', 'content': f'tell me about the word {word} in 50 words'}
    response = client.chat(model='phi', messages=[message])
    content = response["message"]["content"]
    return content

In [75]:
import uuid

class _DatasetGeneratorPickle:
    def __init__(self, generator, generator_id=None):
        self.generator = generator
        self.generator_id = (
            generator_id if generator_id is not None else str(uuid.uuid4())
        )

    def __call__(self, *args, **kwargs):
        return self.generator(*args, **kwargs)

    def __reduce__(self):
        return (_DatasetGeneratorPickle_raise, (self.generator_id,))


def _DatasetGeneratorPickle_raise(*args, **kwargs):
    raise AssertionError("cannot actually unpickle _DatasetGeneratorPickle!")

In [76]:
def my_generator(n, *args, **kwargs):
    for i in range(n):
        content = data_from_phi()
        
        yield {"col_1": content}

my_iterable_dataset = IterableDataset.from_generator(_DatasetGeneratorPickle(my_generator), gen_kwargs={"n": 10})
for example in my_iterable_dataset:
    print(example)

{'col_1': " Constraints or boundaries, limits, or restrictions on something. They can be physical limitations (like a fence) or mental/emotional barriers (like fear). Constraints can be both positive and negative - they can limit our choices but also help us focus on what's important. #Constraints\n"}
{'col_1': " Conven is a term that refers to ease of use or comfortability. It can also mean a state of being comfortable or at ease, such as when everything is organized and easy to access. In general, it's used to describe something that makes life easier or more comfortable for the user. #Convenience #EaseOfUse\n"}
{'col_1': ' Subdivision is the process of dividing something into smaller parts or sections. It can refer to physical land divisions, such as parcels of land divided for residential or commercial use, or it can refer to language, where words are divided into syllables or phonemes. In computer science, it can also refer to breaking up data into smaller pieces for efficient sto